In [5]:
from pyspark.sql import SQLContext
#from pyspark.sql import SQLContext, functions as F, SparkSession

from pyspark.sql import SparkSession, functions as F
from pyspark import  SQLContext
   
spark_session = SparkSession.builder.appName('P8').getOrCreate()
# Si besoin d'effectuer des requetes SQL sur un DF:
# sc = spark_session.sparkContext
# sqlContext = SQLContext(sc)

In [6]:
dept = [("Finance",10),
        ("Marketing",20),
        ("Sales",30),
        ("IT",40)
      ]


deptColumns = ["dept_name","dept_id"]
deptDF = spark_session.createDataFrame(data=dept, schema = deptColumns)
deptDF.printSchema()
deptDF.show(truncate=False)


deptDF.show(10)

root
 |-- dept_name: string (nullable = true)
 |-- dept_id: long (nullable = true)

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|Finance  |10     |
|Marketing|20     |
|Sales    |30     |
|IT       |40     |
+---------+-------+

+---------+-------+
|dept_name|dept_id|
+---------+-------+
|  Finance|     10|
|Marketing|     20|
|    Sales|     30|
|       IT|     40|
+---------+-------+



In [7]:
# https://spark.apache.org/docs/latest/ml-features.html
from pyspark.ml.feature import PCA
from pyspark.ml.linalg import Vectors

data = [(Vectors.sparse(5, [(1, 1.0), (3, 7.0)]),),
        (Vectors.dense([2.0, 0.0, 3.0, 4.0, 5.0]),),
        (Vectors.dense([4.0, 0.0, 0.0, 6.0, 7.0]),)]
df = spark_session.createDataFrame(data, ["features"])

pca = PCA(k=3, inputCol="features", outputCol="pcaFeatures")
model = pca.fit(df)

result = model.transform(df).select("pcaFeatures")
result.show(truncate=False)


+-----------------------------------------------------------+
|pcaFeatures                                                |
+-----------------------------------------------------------+
|[1.6485728230883807,-4.013282700516296,-5.524543751369388] |
|[-4.645104331781534,-1.1167972663619026,-5.524543751369387]|
|[-6.428880535676489,-5.337951427775355,-5.524543751369389] |
+-----------------------------------------------------------+



In [8]:
import os
import boto3

from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession

import sagemaker
from sagemaker import get_execution_role
import sagemaker_pyspark

role = get_execution_role()

# Configure Spark to use the SageMaker Spark dependency jars
jars = sagemaker_pyspark.classpath_jars()

classpath = ":".join(sagemaker_pyspark.classpath_jars())

# See the SageMaker Spark Github to learn how to connect to EMR from a notebook instance
spark = (
    SparkSession.builder.config("spark.driver.extraClassPath", classpath)
    .master("local[*]")
    .getOrCreate()
)

spark

In [9]:
# necessary import 
from pyspark.sql import SparkSession
from pyspark.ml.image import ImageSchema
from pyspark.sql.functions import lit
from functools import reduce
# loaded image
images = ImageSchema.readImages("/home/ec2-user/data")


In [20]:
import pyspark
def readImages(path,
               sparkSession = None,         #TODO: do we need this parameter in Python signature?
               recursive = True,
               numPartitions = 0,
               dropImageFailures = False,
               sampleRatio = 1.0):
    """
    Reads the directory of images from the local or remote (WASB) source.
    Example: spark.readImages(path, recursive = True)
    Args:
        path (str): Path to the image directory
        sparkSession (SparkSession): Existing sparkSession
        recursive (bool): Recursive search flag
        numPartitions (int): Number of the dataframe partitions
        dropImageFailures (bool): Drop the files that are not valid images from the result
        sampleRatio (double): Fraction of the images loaded
    Returns:
        DataFrame: DataFrame with a single column of "images", see imageSchema
        for details
    """
    ctx = SparkContext.getOrCreate()
    imageSchema = ctx._jvm.org.apache.spark.image.ImageSchema
    sql_ctx = pyspark.SQLContext.getOrCreate(ctx)
    jsession = sql_ctx.sparkSession._jsparkSession
    jresult = imageSchema.readImages(path, jsession, recursive, numPartitions, dropImageFailures, float(sampleRatio))
    return DataFrame(jresult, sql_ctx)


readImages("/home/ec2-user/data/fruits-360/Training/*",spark_session,True,0,False,1.0)


TypeError: 'JavaPackage' object is not callable